# Crossmatch catalogs

In [1]:
"""
    TITLE   : x-match
    AUTHOR  : Nathaniel Starkman
    PROJECT :
""";

__author__ = ''
__version__ = ''

<span style='font-size:30px;font-weight:650'>
    About
</span>

This is a dev notebook, where the xmatch functions from `starkman_thesis` are built and tested


<br><br>

- - - 


## Prepare

### Imports

In [2]:
from utilipy import ipython

# ipython.run_imports(base=True, astropy=True, matplotlib=True, verbose_imports=False)
# %run "utilipy.imports.astropy_imports"

# BUILT-IN

# THIRD PARTY

from astropy import table
from astropy import coordinates as coords, units as u
from astropy.coordinates import SkyCoord, Distance, EarthLocation, match_coordinates_sky
from astropy.coordinates.tests.utils import randomly_sample_sphere
from astropy.time import Time

import numpy as np

# PROJECT-SPECIFIC

import starkman_thesis

set autoreload to 1
/Users/nathanielstarkman/miniconda3/lib/python3.7/site-packages/astropy/coordinates/builtin_frames/galactocentric.py:373: AstropyDeprecationWarning: In v4.1 and later versions, the Galactocentric frame will adopt default parameters that may update with time. An updated default parameter set is already available through the astropy.coordinates.galactocentric_frame_defaults ScienceState object, as described in but the default is currently still set to the pre-v4.0 parameter defaults. The safest way to guard against changing default parameters in the future is to either (1) specify all Galactocentric frame attributes explicitly when using the frame, or (2) set the galactocentric_frame_defaults parameter set name explicitly. See http://docs.astropy.org/en/latest/coordinates/galactocentric.html for more information.
  AstropyDeprecationWarning)



<br><br>

- - - 


## Discrete Matches

In [194]:
cat0 = table.Table()
cat0["color"] = ["blue"] * 4 + ["green"] * (10 - 5) + ["orange"]
cat0["id"] = ["tag1"] * 1 + ["tag2"] * (len(cat0["color"]) - 2) + ["tag3"]
cat0["id2"] = ["tag1"] * 10

cat1 = table.Table()
cat1["color"] = ["blue"] * 2 + ["orange"] * 3 + ["green"] * (12 - 6) + ["purple"]
cat1["id"] = ["tag1"] * 4 + ["tag2"] * (len(cat1["color"]) - 6) + ["tag3"]*2
cat1["id2"] = ["tag2"] + ["tag1"] * 11

cat0.to_pandas().values.T
cat1.to_pandas().values.T


array([['blue', 'blue', 'blue', 'blue', 'green', 'green', 'green',
        'green', 'green', 'orange'],
       ['tag1', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2',
        'tag2', 'tag3'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1',
        'tag1', 'tag1']], dtype=object)

array([['blue', 'blue', 'orange', 'orange', 'orange', 'green', 'green',
        'green', 'green', 'green', 'green', 'purple'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag2', 'tag2', 'tag2', 'tag2',
        'tag2', 'tag2', 'tag3', 'tag3'],
       ['tag2', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1',
        'tag1', 'tag1', 'tag1', 'tag1']], dtype=object)

In [95]:
u0 = np.unique(np.array(cat0["color"]))

# first do with only fields=["color"]
# will weed out all stuff in cat1 that's not in cat0
idx10 = cat1["color"] == u0[:, None]

idx0 = True
idx1 = np.sum(idx10, axis=0, dtype=bool)
idx1

# cat0[idx0]
np.shape(cat1[idx1])  # should be N-1=11
cat1[idx1].to_pandas().values.T

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False])

(11,)

array([['blue', 'blue', 'orange', 'orange', 'orange', 'green', 'green',
        'green', 'green', 'green', 'green'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag2', 'tag2', 'tag2', 'tag2',
        'tag2', 'tag2', 'tag3'],
       ['tag2', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1',
        'tag1', 'tag1', 'tag1']], dtype=object)

In [96]:
# Now all fields
u1 = np.unique(np.array(cat0["id"]))
u2 = np.unique(np.array(cat0["id2"]))

idx11 = cat1["id"] == u1[:, None]
idx12 = cat1["id2"] == u2[:, None]

idx0 = True
idx1 = np.sum(idx10 & idx11 & idx12, axis=0, dtype=bool)

np.shape(cat1[idx1])  # should be N-1=11
cat1[idx1].to_pandas().values.T


(6,)

array([['blue', 'green', 'green', 'green', 'green', 'green'],
       ['tag1', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1']], dtype=object)

but the problem is that there are stuff in catalog 0 that are not in catalog 1
Now that we've sifted out all the stuff that are in catalog 1 that are not in 0,
we need to do the reverse.

The general procedure is

(cat0 <- cat1, 2, 3, 4, 5) comparison
cat1 <- cat0 copmarison to filter out non-matches in cat0


In [204]:
import functools
def _match_two_catalogs_indices(match_from, other, fields: list):
    """
    
    Parameters
    ----------
    match_from : Table
        the catalog to match from, only values in the field (from `fields`) are
        permitted in the
    other : Table
        match this against `match_from`
    fields : list
        list of fields to match

    Notes
    -----
    try more axes tricks to avoid for loops (``cat1.to_pandas().values[:, :, None]``)
    the difficulty is that the "unique" arrays can be different lengths
    but these can be extended to be equal length, filled with empty value
    
    """
    # uniques for each field
    uns = (np.unique(np.array(match_from[n])) for n in fields)

    # indices into cat1
    # loop over fields  TODO vectorize ?
    idxs = (other[n] == un[:, None] for n, un in zip(fields, uns))
    # get combined index
    idx = np.sum(functools.reduce(np.logical_and, idxs), axis=0, dtype=bool)

    return idx

def match_catalogs_indices(*catalogs, fields):
    idxs = [_match_two_catalogs_indices(catalogs[0], c, fields=fields)
            for c in catalogs[1:]]  # TODO with numpy
    # and need to do once in reverse on the source catalog
    idx0 = _match_two_catalogs(catalogs[1][idxs[0]], catalogs[0], fields)
    idxs.insert(0, idx0)

    return idx

def match_catalogs(*catalogs, fields):
    idxs = match_catalogs_indices(*catalogs, fields)
    cat_matches = [catalog[idx] for catalog, idx in zip(catalogs, idxs)]
    return cat_matches
# /def

idx = match_catalogs_indices(cat0, cat1, cat2, fields=["color", "id", "id2"])

cat0.to_pandas().values.T
cat0[idx[0]].to_pandas().values.T
# cat1.to_pandas().values.T
# cat1[idx[1]]
# cat2[idx[2]]

array([['blue', 'blue', 'blue', 'blue', 'green', 'green', 'green',
        'green', 'green', 'orange'],
       ['tag1', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2',
        'tag2', 'tag3'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1',
        'tag1', 'tag1']], dtype=object)

array([['blue', 'green', 'green', 'green', 'green', 'green'],
       ['tag1', 'tag2', 'tag2', 'tag2', 'tag2', 'tag2'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1']], dtype=object)

Now 3 catalogs

In [98]:
cat2 = table.Table()
cat2["color"] = ["blue"] * 2 + ["orange"] * 4 + ["purple"] + ["green"] * (12 - 7)
cat2["id"] = ["tag1"] * 4 + ["tag2"] * (len(cat1["color"]) - 6) + ["tag3"]*2
cat2["id2"] = ["tag2"] + ["tag1"] * 11

cat2.to_pandas().values.T

array([['blue', 'blue', 'orange', 'orange', 'orange', 'orange', 'purple',
        'green', 'green', 'green', 'green', 'green'],
       ['tag1', 'tag1', 'tag1', 'tag1', 'tag2', 'tag2', 'tag2', 'tag2',
        'tag2', 'tag2', 'tag3', 'tag3'],
       ['tag2', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1', 'tag1',
        'tag1', 'tag1', 'tag1', 'tag1']], dtype=object)

In [101]:
# just going against catalog 1
idx20 = cat2["color"] == u0[:, None]
idx21 = cat2["id"] == u1[:, None]
idx22 = cat2["id2"] == u2[:, None]
idx2 = np.sum(idx20 & idx21 & idx22, axis=0, dtype=bool)

# lets also go against catalog 2

cat2[idx2].to_pandas().values.T

array([['blue', 'green', 'green', 'green'],
       ['tag1', 'tag2', 'tag2', 'tag2'],
       ['tag1', 'tag1', 'tag1', 'tag1']], dtype=object)

In [102]:
t = [idx20, idx21, idx22]
t

[array([[ True,  True, False, False, False, False, False, False, False,
         False, False, False],
        [False, False, False, False, False, False, False,  True,  True,
          True,  True,  True],
        [False, False,  True,  True,  True,  True, False, False, False,
         False, False, False]]),
 array([[ True,  True,  True,  True, False, False, False, False, False,
         False, False, False],
        [False, False, False, False,  True,  True,  True,  True,  True,
          True, False, False],
        [False, False, False, False, False, False, False, False, False,
         False,  True,  True]]),
 array([[False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True]])]

In [ ]:
raise ValueError

## Code

Observation Time

In [ ]:
ra, dec, _ = randomly_sample_sphere(1000)
pm_ra_cosdec=np.ones_like(ra.value) * 10 * u.mas / u.yr
pm_dec=np.ones_like(dec.value) * u.mas / u.yr
time = Time('2001-03-22 00:01:44.732327132980', scale='utc')
data = coords.ICRS(ra=ra, dec=dec, pm_ra_cosdec=pm_ra_cosdec, pm_dec=pm_dec)
cs = SkyCoord(data, obstime=time)
cs[:10]

Compute the position of the source represented by this coordinate object
to a new time using the velocities stored in this object and assuming
linear space motion (including relativistic corrections). This is
sometimes referred to as an "epoch transformation."

The initial time before the evolution is taken from the ``obstime``
attribute of this coordinate.  Note that this method currently does not
support evolving coordinates where the *frame* has an ``obstime`` frame
attribute, so the ``obstime`` is only used for storing the before and
after times, not actually as an attribute of the frame. Alternatively,
if ``dt`` is given, an ``obstime`` need not be provided at all.

In [ ]:
res = SkyCoord.apply_space_motion(cs, new_obstime=Time.now())
res[:10]

## Example

In [ ]:
result_tgas = dict(ra=66.44280212823296,
                   dec=-69.99366255906372,
                   parallax=22.764078749733947,
                   pmra=144.91354358297048,
                   pmdec=5.445648092997134,
                   ref_epoch=2015.0,
                   phot_g_mean_mag=7.657174523348196)

result_2mass = dict(RAJ2000=[66.421970000000002, 66.433521999999996,
                             66.420564999999996, 66.485068999999996,
                             66.467928999999998, 66.440815000000001,
                             66.440454000000003],
                    DEJ2000=[-70.003722999999994, -69.990768000000003,
                             -69.992255999999998, -69.994881000000007,
                             -69.994926000000007, -69.993613999999994,
                             -69.990836999999999],
                    Jmag=[16.35, 13.663, 16.171, 16.184, 16.292,
                          6.6420002, 12.275],
                    Hmag=[15.879, 13.955, 15.154, 15.856, 15.642,
                          6.3660002, 12.185],
                    Kmag=[15.581, 14.238, 14.622, 15.398, 15.123,
                          6.2839999, 12.106],
                    Date=['1998-10-24', '1998-10-24', '1998-10-24',
                          '1998-10-24', '1998-10-24', '1998-10-24',
                          '1998-10-24'])

In [ ]:
c = SkyCoord(ra=result_tgas['ra'] * u.deg,
             dec=result_tgas['dec'] * u.deg,
             distance=Distance(parallax=result_tgas['parallax'] * u.mas),
             pm_ra_cosdec=result_tgas['pmra'] * u.mas/u.yr,
             pm_dec=result_tgas['pmdec'] * u.mas/u.yr,
             obstime=Time(result_tgas['ref_epoch'], format='decimalyear')
)

In [ ]:
catalog_2mass = SkyCoord(ra=result_2mass['RAJ2000'] * u.deg,
                         dec=result_2mass['DEJ2000'] * u.deg,
                         obstime=Time(result_2mass['Date'][0]))

In [ ]:
c_2mass_epoch = SkyCoord.apply_space_motion(c, catalog_2mass.obstime)
c_2mass_epoch, c_2mass_epoch.obstime

In [ ]:
starkman_thesis.utils.data.xmatch.xmatch_coords(c_2mass_epoch, catalog_2mass)  # obstime=Time.now()

In [ ]:
x = table.Table({k: getattr(catalog_2mass, k) for k in catalog_2mass.get_representation_component_names().keys()})
del x["distance"]
x.meta["obstime"] = catalog_2mass.obstime

In [ ]:
c1, c2, info = starkman_thesis.utils.data.xmatch.xmatch_coords(c, x, obstime=x.meta["obstime"])

## Problems With Duplicates

In [ ]:
sc1 = coords.ICRS(*randomly_sample_sphere(10, randomseed=0))
sc2 = coords.ICRS(*randomly_sample_sphere(12, randomseed=1))

In [ ]:
cat1 = table.Table()
cat1["ra"] = sc1.ra
cat1["dec"] = sc1.dec
cat1["color"] = ["blue"] * 4 + ["green"] * (len(sc1) - 5) + ["orange"]

cat2 = table.Table()
cat2["ra"] = sc2.ra
cat2["dec"] = sc2.dec
cat2["color"] = ["blue"] * 2 + ["orange"] * 3 + ["green"] * (len(sc2) - 5)

cat1
cat2

In [ ]:
mc1, mc2, info = starkman_thesis.utils.data.xmatch.xmatch(cat1, cat2, maxdist=40*u.deg, match_fields="color")
mc1

In [ ]:
field = "color"
values_cat1 = np.array(cat1[field])
len_vc1 = len(values_cat1)
values_cat2 = np.array(cat2[field])
len_vc2 = len(values_cat2)

uniques = np.unique(values_cat1)

# sep = np.ones(len_vc1) * -1.0
# idx1 = -np.ones(len_vc1, dtype=int)
# idx2 = -np.ones(len_vc2, dtype=int)
idx1 = []
idx2 = []
info = {"sep2d": [], "dist3d": []}
for unique in uniques:
    print(unique)
    idx_1 = np.arange(len_vc1)[values_cat1 == unique]
    idx_2 = np.arange(len_vc2)[values_cat2 == unique]

    # the case where a class only exists in one but not the other
    if (idx_1.shape[0] == 0 or idx_2.shape[0] == 0):
        continue
        
    temp_sc1_idx, temp_sc2_idx, temp_info = starkman_thesis.utils.data.xmatch.xmatch_coords_indices(sc1[idx_1], sc2[idx_2], maxdist=40*u.deg)

    idx1.extend(idx_1[temp_sc1_idx])
    idx2.extend(idx_2[temp_sc2_idx])

    info["sep2d"].extend(temp_info.get("sep2d", []))
    info["dist3d"].extend(temp_info.get("dist3d", []))
    print(idx_1[temp_sc1_idx])

idx1 = np.array(idx1)
idx1 = idx1[idx1>=0]
idx2 = np.array(idx2)
idx2 = idx2[idx2>=0]
info["sep2d"] = coords.Angle(info["sep2d"])
info["dist3d"] = u.Quantity(info["dist3d"])

info
cat1[idx1]
cat1[idx1]

In [ ]:
i, sep2d, sep3d = coords.match_coordinates_sky(sc1[idx_1], sc2[idx_2])
midx = np.where(sep2d < 40 * u.deg)[0]

sc1[idx_1][midx].separation(sc2[idx_2][i][midx])

<br><br>

- - - 
- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>